### Import des bibliothéques
---

In [3]:
import pandas as pd
import requests as rq
import json as js


### Données du projet : Top 35 Villes
---

In [4]:
Top_ville = [
    "Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen",
    "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg",
    "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy",
    "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis",
    "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes",
    "Aigues Mortes", "Saintes Maries de la Mer", "Collioure",
    "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz",
    "Bayonne", "La Rochelle"
]


### Scraping des coordonnées GPS
---

In [5]:
URL = 'https://nominatim.openstreetmap.org/search'
headers = {'User-Agent': 'MonProjetKayak/1.0'}
params = {'city': 'St Malo', 'country': 'France', 'format': 'json'}

# Test sur St Malo
response=rq.get(
    URL,
    params=params,
    headers=headers
    )

response.json()


[{'place_id': 263088376,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 905534,
  'lat': '48.6495180',
  'lon': '-2.0260409',
  'class': 'boundary',
  'type': 'administrative',
  'place_rank': 16,
  'importance': 0.6170333686756257,
  'addresstype': 'town',
  'name': 'Saint-Malo',
  'display_name': 'Saint-Malo, Ille-et-Vilaine, Bretagne, France métropolitaine, 35400, France',
  'boundingbox': ['48.5979853', '48.6949736', '-2.0765246', '-1.9367259']},
 {'place_id': 261977249,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 1653637,
  'lat': '48.4904728',
  'lon': '-1.7421555',
  'class': 'boundary',
  'type': 'administrative',
  'place_rank': 14,
  'importance': 0.5038359135553251,
  'addresstype': 'municipality',
  'name': 'Saint-Malo',
  'display_name': 'Saint-Malo, Ille-et-Vilaine, Bretagne, France métropolitaine, France',
  'boundin

In [6]:
# GPS top 35 Villes
df_ville = pd.DataFrame(columns=['Ville', 'Latitude', 'Longitude'])
params={'city': 'ville', 'country': 'France', 'format': 'json'}

for ville in Top_ville:
    print(ville)
    try:
        response = rq.get(
            URL,
            params=params, 
            headers=headers,
            timeout=10 # si nominatim ne répond pas s'arrête ds les 10s
        )
        response.raise_for_status()
        data = response.json()

        if data:  # seulement si des données sont présentes
            lat = data[0]['lat']
            lon = data[0]['lon']

            # Ajout dans le DataFrame uniquement si coordonnées disponibles
            df_ville.loc[len(df_ville)] = [ville, lat, lon]

        else:
            print(f"\tAucune donnée trouvée pour {ville}")  # pas ajouté

    except rq.RequestException as e:
        print(f"\tErreur pour {ville} : {e}")  # pas ajouté

df_ville.head()


Mont Saint Michel
St Malo
Bayeux
Le Havre
Rouen
Paris
Amiens
Lille
Strasbourg
Chateau du Haut Koenigsbourg
Colmar
Eguisheim
Besancon
Dijon
Annecy
Grenoble
Lyon
Gorges du Verdon
Bormes les Mimosas
Cassis
Marseille
Aix en Provence
Avignon
Uzes
Nimes
Aigues Mortes
Saintes Maries de la Mer
Collioure
Carcassonne
Ariege
Toulouse
Montauban
Biarritz
Bayonne
La Rochelle


,Ville,Latitude,Longitude
0,Mont Saint Michel,49.5532646,2.9392577
1,St Malo,49.5532646,2.9392577
2,Bayeux,49.5532646,2.9392577
3,Le Havre,49.5532646,2.9392577
4,Rouen,49.5532646,2.9392577


### Sauvegarde df_ville
---

In [ ]:
df_ville.to_csv('df_ville.csv')